In [1]:
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta 

In [2]:
data = pd.read_csv("Invoice_20201220.csv", sep="|")
data["SITE_NAME"] = data["SITE_NAME"].str.extract(r'\((.*?)\)')

In [3]:
data = data.drop(["INVOICE_NAME"], axis=1)

In [4]:
data["END"] = pd.to_datetime(data["END"], format='%d/%m/%Y')
data["BEGIN"] = pd.to_datetime(data["BEGIN"], format='%d/%m/%Y')
data = data.assign(DURATION=data["END"].sub(data["BEGIN"]))
data["DURATION"] = data["DURATION"].dt.days

In [5]:
data = data.assign(MEAN_CONSO_BY_DAY=data["KWH_PERIOD"].str.replace(" ","").astype(int)/(data["DURATION"]))

In [13]:
all_site_number = data["SITE_NAME"].value_counts()
data = data.assign(TOTAL_BILL_BY_SITE= data["SITE_NAME"].map(all_site_number))

1600BR    105
1089BR     70
4060BR     70
2079BR     70
0038BB     37
         ... 
1326BR      1
1341BR      1
1347BR      1
0670BR      1
1203KO      1
Name: SITE_NAME, Length: 2403, dtype: int64

In [7]:
site_name_mean_list = data.groupby("SITE_NAME")["MEAN_CONSO_BY_DAY"].mean()
site_name_sigma_list = data.groupby("SITE_NAME")["MEAN_CONSO_BY_DAY"].std(ddof=0)
site_name_df = data["SITE_NAME"].to_frame().drop_duplicates()
site_name_df = site_name_df.assign(MEAN_BY_SITE=site_name_df["SITE_NAME"].map(site_name_mean_list))
site_name_df = site_name_df.assign(STD_BY_SITE=site_name_df["SITE_NAME"].map(site_name_sigma_list))
site_name_df = site_name_df.assign(COV= site_name_df["STD_BY_SITE"] / site_name_df["MEAN_BY_SITE"])
site_name_df = site_name_df.drop(["MEAN_BY_SITE", "STD_BY_SITE"], axis = 1)
display(site_name_df)

,SITE_NAME,COV
0,0001BB,0.065117
35,0001BR,0.049759
70,0001KO,0.102968
105,0002BB,0.065880
140,0002BR,0.082812
...,...,...
51423,1486BR,0.000000
51424,2086BR,0.442294
51426,1179BB,0.000000
51427,0632KO,0.011349


In [196]:
data.to_csv("./data.csv")
site_name_df.to_csv("./site_name_df.csv")